In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import torch
# Add the src directory to the path. TEMPORARY FIX
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../..")))

from models.duck_net.duck_net import DuckNet
from src.models.data_management.cnn_formes import CNNFormes
from src.data_processing.dataset_loader import CoastData
from src.models.metrics import Metrics

from src.models.data_management.data_loader import DataLoaderManager

import cv2

In [ ]:
artifact_path = os.path.abspath(os.path.join(os.getcwd(), "../../../artifacts/"))

num_classes = 3
margin = 10
metrics = {
    "train": Metrics(phase="train", num_classes=num_classes, average=None, use_margin=False, margin = margin),
    "validation": Metrics(phase="val", num_classes=num_classes, average=None, use_margin=False, margin = margin),
    "test": Metrics(phase="test", num_classes=num_classes, average='micro', use_margin=True, margin = margin) # 'macro'
}

# Load the data to split it and save it to a dict
duck = DuckNet(num_classes=3, experiment_name="test_experiments", use_mlflow=False)

duck.load_model(os.path.abspath(os.path.join(artifact_path, "2025-03-02-16-06-29/models/best_model.pth")))

data_path = os.path.abspath(os.path.join(os.getcwd(), "../../../data/processed_4_classes"))

coast_data = CoastData(data_path)
split = coast_data.split_data()

data = DataLoaderManager.load_data(split)

In [ ]:
# for split in data:
split = "test"
print(f"Split: {split}")
for img_path, mask_path in zip(data[split]["images"], data[split]["masks"]):
    pred = duck.predict_patch(img_path, combination="max")
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask = torch.from_numpy(mask).float() 
    metrics[split].update_metrics(pred, mask)

metrics[split].compute()
print(metrics[split].get_last_epoch_info())